In [6]:
!pip install torch
!pip install torchtyping
!pip install stltovoxel

In [4]:
#STL File Preprocessing
import stltovoxel
import matplotlib
import os
import torch
import torch.nn as nn
from torchtyping import TensorType
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


# Define the folder containing the STL files
stl_folder = 'stl'
output_folder = 'voxel'
resolution = 100
target_width = 100
target_height = 100

stl = []
label = [1, 1, 2]

for filename in os.listdir(stl_folder):
    if filename.lower().endswith('.stl'):
        stl_path = os.path.join(stl_folder, filename)
        output_voxel_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '_voxels.png')
        stltovoxel.convert_file(stl_path, output_voxel_path, resolution)
        voxel_graph = np.load(output_voxel_path)
        voxel_tensor = torch.tensor(voxel_graph, dtype=torch.float32)
        stl.append(voxel_tensor.flatten())


label = torch.tensor(label, dtype=torch.long)
stl = torch.stack(stl)

dataset = TensorDataset(stl, label)
data_loader = DataLoader(dataset, batch_size=1)

exception (False, 'No lines found, impossible to read')
Processing layer 0/6
Processing layer 1/6
Processing layer 2/6
Processing layer 3/6
Processing layer 4/6
export png 0/8
export png 1/8
export png 2/8
export png 3/8
export png 4/8
export png 5/8
export png 6/8
export png 7/8


FileNotFoundError: [Errno 2] No such file or directory: 'voxel\\Cube10x10x10_voxels.png'

In [25]:
#Network Block



#Input in the form of a voxelized stl file
#Output is a number corresponding to a Solidworks Feature
#Ex: 0 is none, 1 is boss, 2 is revolve, etc(rest is in notes)


torch.manual_seed(0)
class FeatureRecognition(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        self.first_linear = nn.Linear(784, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.projection = nn.Linear(512, 3)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, stl: TensorType[float]) -> TensorType[float]:
        torch.manual_seed(0)
        stl = stl.flatten()
        out = self.sigmoid(self.projection(self.dropout(self.relu(self.first_linear(stl)))))
        return torch.round(out, decimals=4)

In [ ]:
#Traning block

model = FeatureRecognition()

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
dataSetSize = 3

#Traning loop
epochs = 5
for i in range(epochs):
    for stl, label in data_loader:
        stl = stl.view(stl.shape[0], 784)           #FIX

        modelPrediction = model(stl)
        optimizer.zero_grad()
        loss = loss_function(modelPrediction, label)
        loss.backward()
        optimizer.step()

In [ ]:
#Evaluation Block

model.eval()

stl = stl.view(stl.shape[0], 780)               #FIX

modelPrediction = model(stl)
max = torch.max(modelPrediction, dim = 1)
for i in range(len(stl)):
    print(max[i].item())
